### Buy AAPL when GOOG rallies 5% in the last 5 minutes

This is a good example of a quantitative trading strategy. Since we expect AAPL and GOOG to move together, we want to buy AAPL when GOOG rallies.

You want to track GOOG’s price and execute an order in AAPL. To do this, subscribe to GOOG’s tick data and store it in a DataFrame. Then you can submit an order.

Start with the imports.

In [1]:
import pandas as pd
import threading
import time

from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import *

In the class that creates the IB app, we include a dictionary to store GOOG’s data. 

In [2]:
class IBapi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.bardata = {}  # Initialize dictionary to store bar data

    def nextValidId(self, orderId: int):
        super().nextValidId(orderId)
        self.nextorderId = orderId
        print("The next valid order id is: ", self.nextorderId)

    def tick_df(self, reqId, contract):
        """custom function to init DataFrame and request Tick Data"""
        self.bardata[reqId] = pd.DataFrame(columns=["time", "price"])
        self.bardata[reqId].set_index("time", inplace=True)
        self.reqTickByTickData(reqId, contract, "Last", 0, True)
        return self.bardata[reqId]

    def tickByTickAllLast(
        self,
        reqId,
        tickType,
        time,
        price,
        size,
        tickAtrribLast,
        exchange,
        specialConditions,
    ):
        if tickType == 1:
            self.bardata[reqId].loc[pd.to_datetime(time, unit="s")] = price

    def stock_contract(
        self,
        symbol,
        secType='STK',
        exchange='SMART',
        currency='USD'
    ):
        # create a stock contract
        contract = Contract()
        contract.symbol = symbol
        contract.secType = secType
        contract.exchange = exchange
        contract.currency = currency

        return contract

The class has a custom function called `tick_df`. This function creates a DataFrame and adds it to the dictionary you created. It then populates the DataFrame with data. If you’re wondering where I define `reqTickByTickData`, it’s from the `EClient` class. Calling this function starts the data stream. Finally, the `tickByTickAllLast` function returns the last price. 

The code in `tickByTickAllLast` comes from pandas. The `.loc` method specifies the row and column that we want to insert data into. This line inserts the last price into a row. That row’s index is the current timestamp.

Finally, add the `stock_contract` function into the class. Don’t forget to add `self` as the first argument.

Next, create the function to start the app and submit an order.

In [3]:
def run_loop():
    app.run()


def submit_order(
    contract, 
    direction, 
    qty=100, 
    ordertype="MKT", 
    transmit=True
):
    # Create order object
    order = Order()
    order.action = direction
    order.totalQuantity = qty
    order.orderType = ordertype
    order.transmit = transmit
    order.eTradeOnly = ""
    order.firmQuoteOnly = ""
    # submit order
    app.placeOrder(app.nextorderId, contract, order)
    app.nextorderId += 1


def check_for_trade(df, contract):
    start_time = df.index[-1] - pd.Timedelta(minutes=5)
    min_value = df[start_time:].price.min()
    max_value = df[start_time:].price.max()

    if df.price.iloc[-1] < max_value * 0.95:
        submit_order(contract, "SELL")
        return True

    elif df.price.iloc[-1] > min_value * 1.05:
        submit_order(contract, "BUY")
        return True

The `check_for_trade` function is where the logic is. It decides whether a trade is made or not. It takes the last price and subtracts five minutes from the time we received it. Use the `min()` and `max()` functions from pandas to determine the high and low over the last five minutes.

With those values, it checks if the current price is 5% greater than or less than the min or max. If so, the function submits an order and returns `True` to let you know the order was sent.

Next, run the algorithm.

In [4]:
app = IBapi()
app.nextorderId = None
app.connect("127.0.0.1", 7496, 123)


api_thread = threading.Thread(target=run_loop)
api_thread.start()


while True:
    if isinstance(app.nextorderId, int):
        print("connected")
        break
    else:
        print("waiting for connection")
        time.sleep(1)


goog = app.stock_contract("GOOG")
aapl = app.stock_contract("AAPL")

ERROR -1 2104 Market data farm connection is OK:uscrypto
ERROR -1 2104 Market data farm connection is OK:eufarm
ERROR -1 2104 Market data farm connection is OK:usopt
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


waiting for connection
The next valid order id is:  2
connected


Create the app and connect to IB. Then check that the API is connected. Finally, use our custom functions to create GOOG and AAPL contracts.

Next, we get market data for GOOG. Then we check that the data is flowing. At this point, we have the data but we need five minutes of history to run the algorithm. Subtracting two timestamps from each other returns a timedelta object. timedelta has a seconds method which gives us the number of seconds between two times. We need at least 5 minutes, or 300 seconds, worth of data. So we will put the script to sleep for 300 seconds minus whatever time has already elapsed.

In [ ]:
df = app.tick_df(401, goog)


time.sleep(10)
for i in range(100):
    if len(df) > 0:
        break
    time.sleep(0.3)


if i == 99:
    app.disconnect()
    raise Exception("Error with Tick data stream")


data_length = df.index[-1] - df.index[0]
if data_length.seconds < 300:
    time.sleep(300 - data_length.seconds)

Now, find trades!

In [ ]:
while True:
    if check_for_trade(df, aapl):
        break
    time.sleep(0.1)

app.disconnect()

This runs on an infinite loop. If a trade is found (remember `check_for_trade` returns `True`) then exit the loop. Otherwise, wait 100 milliseconds and check again.